In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.colors
import statsmodels.formula.api as smf
import seaborn as sns
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import statsmodels.api as sm


In [63]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df['Country Code']=pop_df['Country Code'].apply(lambda x: x.lower())
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ', 
    'PRK', #North Korea
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [64]:
#Attention
df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df.rename(columns={'aggregated_value': 'Attention_count', 'country': 'Target', 'affiliation_country': 'Source'}, inplace=True)
df = df[df['year'].isin(np.arange(2002, 2020))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]

df=df.groupby(['year', 'Source', 'Target'])['Attention_count'].sum().reset_index()


In [65]:
#funding

physical_sciences=[ 'MATH', 'ENGI', 'PHYS', 'COMP','MULT']
df_fund=pd.read_csv(r"C:\Users\Yasaman\Downloads\Mentioned-Funding-fractional counting.csv")
df_fund.rename(columns={'aggregated_value': 'Funding_count', 'country': 'Target', 'funding_country': 'Source'}, inplace=True)
df_fund=df_fund[~(df_fund['subjarea'].isin(physical_sciences))]
df_fund=df_fund.groupby(['year', 'Source', 'Target'])['Funding_count'].sum().reset_index()
df_fund=df_fund[(df_fund['year'].isin(np.arange(2002,2020,1)))].reset_index(drop=True)

In [72]:
df_mig=pd.read_csv(r"C:\Users\Yasaman\Downloads\scopus_2024_V1_scholarlymigration_countryflows_enriched_new.csv")

df_mig=df_mig[['iso3codefrom','iso3codeto','year','n_migrations']]
df_mig=df_mig[df_mig['year'].isin(np.arange(2002, 2020, 1))]
df_mig['iso3codefrom']=df_mig['iso3codefrom'].apply(lambda x:x.lower())
df_mig['iso3codeto']=df_mig['iso3codeto'].apply(lambda x:x.lower())
df_mig['year']=df_mig['year'].astype(int)
df_mig.rename(columns={'iso3codefrom':'Target', 'iso3codeto':'Source', 'n_migrations':"Migration_count"}, inplace=True)

In [74]:
total_df=df.merge(df_fund, on=['year', 'Source','Target'], how='outer')
total_df=total_df.merge(df_mig, on=['year', 'Source','Target'], how='outer')
total_df=total_df[(total_df['Source'].isin(possible_iso))&(total_df['Target'].isin(possible_iso))].fillna(0).reset_index(drop=True)

In [75]:
total_df['treated_CW']=total_df['Target'].isin(['yem', 'lby', 'syr']).astype(int)
total_df['treated_GO']=total_df['Target'].isin(['egy', 'tun']).astype(int)
total_df['treated_GC']=total_df['Target'].isin(['omn', 'kwt', 'bhr', 'mar','jor']).astype(int)
total_df['post']=total_df['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )
total_df['log_attention']=np.log(total_df['Attention_count']+1)
total_df['log_funding']=np.log(total_df['Funding_count']+1)
total_df['log_migration']=np.log(total_df['Migration_count']+1)

In [71]:
total_df=total_df[total_df['Source']!=total_df['Target']]

In [82]:
# Running the Difference-in-Differences regression
model = smf.ols("log_attention ~ treated_CW * post+treated_GC * post+treated_GO* post +log_funding+ log_migration+C(Source)+C(Target) + C(year)", data=total_df).fit(cov_type='cluster', cov_kwds={'groups': total_df['Target']})
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          log_attention   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     568.7
Date:                Wed, 12 Mar 2025   Prob (F-statistic):          2.57e-157
Time:                        16:11:21   Log-Likelihood:            -1.3607e+05
No. Observations:              161015   AIC:                         2.728e+05
Df Residuals:                  160704   BIC:                         2.759e+05
Df Model:                         310                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.4104      0.094  

c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 314, but rank is 144
  warnings.warn('covariance of constraints does not have full '


In [ ]:
df_mig['year']=df_mig['year'].astype(int)